This notebook is used to download weather data from open meteo

In [11]:
import time
import requests
import pandas as pd
from typing import List, Dict, Optional

#Function to slow down requests to avoid rate limit on the API
def fetch_with_retry(url, params, max_retries=3):
    for _ in range(max_retries):
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            wait_time = int(response.headers.get("Retry-After", 10))
            print(f"Rate limited. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        else:
            print(f"Error {response.status_code}: {response.text}")
            break
    return None


def fetch_historical_weather_multiple(
    latitudes: List[float],
    longitudes: List[float],
    start_datetime: str,
    end_datetime: str,
    location_names: Optional[List[str]] = None,
) -> Dict[str, pd.DataFrame]:
    """
    Fetch historical weather data for multiple lat/lon pairs, including wind direction.

    Args:
        latitudes: List of latitudes.
        longitudes: List of longitudes.
        start_date: Start date in "YYYY-MM-DD" format.
        end_date: End date in "YYYY-MM-DD" format.
        location_names: Optional names for each location (default: "loc_0", "loc_1", ...).

    Returns:
        Dictionary of DataFrames (key: location name, value: weather data).
    """
    start_date = start_datetime.split("T")[0]
    end_date = end_datetime.split("T")[0]

    if len(latitudes) != len(longitudes):
        raise ValueError("Latitudes and longitudes must have the same length.")
    
    if location_names is None:
        location_names = [f"loc_{i}" for i in range(len(latitudes))]
    elif len(location_names) != len(latitudes):
        raise ValueError("Location names must match latitudes/longitudes length.")

    weather_data = {}
    base_url = "https://archive-api.open-meteo.com/v1/archive"

    for lat, lon, name in zip(latitudes, longitudes, location_names):
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": start_date,
            "end_date": end_date,
            "hourly": [
                "temperature_2m",
                "relative_humidity_2m",
                "wind_speed_10m",
                "wind_direction_10m"
            ],
            "timezone": "UTC"
        }
        
        data = fetch_with_retry(base_url, params=params,max_retries=5)

        df = pd.DataFrame(data["hourly"])
        df['time'] = pd.to_datetime(df['time'], utc=True)
        time_filter = (df["time"] >= start_datetime) & (df["time"] <= end_datetime)
        df["location"] = name 
        df = df[time_filter]
        df['time'] = df['time'] - pd.Timedelta(hours=5)
        weather_data[name] = df

    return weather_data

#Adjust for location of wind farm CSV
poi_df = pd.read_csv(r'C:\Users\Kavinda Nissanka\Documents\Erdos Institute\Wind Power Prediction\hydroquebec_wind_farms.csv')
poi_df_inservice_stations = poi_df[poi_df['status']=='In service']
# Example: Montreal (45.5017° N, 73.5673° W)
latitude = poi_df_inservice_stations['latitude']
longitude = poi_df_inservice_stations['longitude']
start_date = "2019-01-01T05:00:00"
end_date = "2024-01-01T04:00:00"

weather_data = fetch_historical_weather_multiple(latitude, longitude, start_date, end_date,location_names=poi_df_inservice_stations['name'])
print(weather_data)

Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
{'Baie-des-Sables wind farm':                            time  temperature_2m  relative_humidity_2m  \
5     2019-01-01 00:00:00+00:00            -7.0                    80   
6     2019-01-01 01:00:00+00:00            -7.1                    80   
7     2019-01-01 02:00:00+00:00            -6.8                    80   
8     2019-01-01 03:00:00+00:00            -6.8                    81   
9     2019-01-01 04:00:00+00:00            -6.8                    81   
...                         ...             ...                   ...   
43824 2023-12-31 19

In [12]:
keys = weather_data.keys()
#Adjust path to wherever you want to save it
for key in keys:
    weather_at_station = pd.DataFrame(weather_data[key])
    weather_at_station.to_csv(r'V:\Erdos Institute Data Downloads\windfarm_weather_data_in_service\{}_hourly_weather_2019-2023.csv'.format(key.replace(' ','_')))